# Shear Likelihood 1.2

### Preparing necessary libraries

In [ ]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import timedelta
from astropy import units
from numpy import random
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units
from scipy.stats import kstest

### Calculating Likelihood

The measured shear $g^i_t$ is given by 

$$ g^i_t = g^{I,i}_t + g^{L,i}_t $$

where $g^{I,i}_t$ is the intrisic shear and $g^{L,i}_t$ is the lensing shear. Assuming that the intrinsic shear is drawn from a normal distribution with average 0 and variance $\sigma_{g^I_t}^2$, the probability of measuring $g^i_t$, given a certain $r$, $z$ and other cosmological parameters $\vec{\theta_c}$ is given by

$$ P(g^i_t | r, z, \vec{\theta_c}) = \frac{1}{\sqrt{2\pi}\sigma_{g^I_t}} \exp\left[\frac{-(g^i_t - g^{L,i}_t(r,z,\vec{\theta_c}))^2}{2\sigma_{g^I_t}^2}\right] $$
Then, to find the likelihood of the set of all $\{g^i_t\}_i^{N}$ measured, we perform the following integral for all $g^i_t$:

$$ P(g^i_t | \vec{\theta_c}) = \int_0^{r_{max}} \int_{z_{cluster}}^{\infty} \mathcal{d}r \mathcal{d}z P(g^i_t | r, z, \vec{\theta_c}) P(r) P(z) $$

where $P(r)$ and $P(z)$ are the distributions for $r$ and $z$.

This is a computationally expensive process though. To avoid it, we bin the data. We then have to perform the same integrals, but for each bin and each shear $g^i_t$ in the bin. At this moment we'll choose to not calculate them, and instead use an estimator for the probability

$$ \tilde{P}(g^i_t | \vec{\theta_c}) = \frac{1}{N_i} \sum_j^{N_i} P(g^i_t | r_j, z_j, \vec{\theta_c}) $$

where $N_i$ is the number of galaxies in the bin. The bin likelihood is then

$$ \mathcal{L}_i = \prod_j^{N_i} \tilde{P}(g^j_t | \vec{\theta_c}) $$

and the total likelihood is 

$$ \mathcal{L} = \prod_i^M \mathcal{L}_i = \prod_i^M \prod_j^{N_i} \tilde{P}(g^j_t | \vec{\theta_c}) =  \prod_i^M \prod_j^{N_i} \left[ \frac{1}{N_i} \sum_k^{N_i} P(g^j_t | r_k, z_k, \vec{\theta_c}) \right]  $$

In [ ]:
def shear_probability(gt_obs, gt_th, sigma):

    return np.exp(-np.power(np.subtract(gt_obs, gt_th), 2)/2/sigma**2)/np.sqrt(2*np.pi)/sigma

In [ ]:
def shear_log_likelihood(logm, gc, radius, shapenoise, cosmo, concentration=4, delta=200, model='nfw'):

    n_bins     = len(gc.profile)
    cluster_z  = gc.z
    m          = float(10.**logm)

    gal_z      = gc.galcat['z']
    gt_th      = clmm.compute_reduced_tangential_shear(radius, m, concentration, cluster_z, gal_z, cosmo, delta_mdef=200, halo_profile_model=model)
    gt_obs     = gc.galcat['et']

    likelihood = 0

    for i in range(n_bins):

        gals_bin    = gc.profile['gal_id'][i]
        gt_th_bin   = [gt_th[gal_id] for gal_id in gals_bin]
        gt_obs_bin  = [gt_obs[gal_id] for gal_id in gals_bin]

        for gt in gt_obs_bin:

            gt_prob = np.mean(shear_probability(gt, gt_th_bin, shapenoise))
            if gt_prob != 0:
                likelihood += np.log(gt_prob)

    return -2*likelihood

In [ ]:
np.random.seed(0)

# Define cosmological parameters
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)
    
cluster_m     = 1.e15 # Cluster mass
cluster_z     = 0.4   # Cluster redshift
concentration = 4     # Concentrion parameter NFW profile
ngals         = 10000 # Number of galaxies
Delta         = 200   # Overdensity parameter definition NFW profile
cluster_ra    = 0.0   # Cluster right ascension
cluster_dec   = 0.0   # Cluster declination
shapenoise    = 1e-3 # True ellipticity standard variation

# Create galaxy catalog and Cluster object
data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, "chang13", zsrc_min = cluster_z + 0.2, shapenoise=shapenoise, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
gc = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, data)

gc.compute_tangential_and_cross_components(geometry="flat")
radius = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)

# Create binning profile por binned methods
bin_edges = da.make_bins(0.7, 4, 50)
profile = gc.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
from clmm.support.sampler import samplers

# radius = convert_units(gc_noisy_z.galcat['theta'], 'radians', 'Mpc', redshift=gc_noisy_z.z, cosmo=cosmo)
logm_0 = random.uniform(13., 17., 1)[0]

logm_est = samplers['basinhopping'](shear_log_likelihood, logm_0, minimizer_kwargs={'args':(gc, radius, shapenoise, cosmo)})

print(logm_est)

In [ ]:
np.random.seed(0)

# Define cosmological parameters
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)
    
cluster_m     = 1.e15 # Cluster mass
cluster_z     = 0.4   # Cluster redshift
concentration = 4     # Concentrion parameter NFW profile
ngals         = 10000 # Number of galaxies
Delta         = 200   # Overdensity parameter definition NFW profile
cluster_ra    = 0.0   # Cluster right ascension
cluster_dec   = 0.0   # Cluster declination
shapenoise    = 1e-2 # True ellipticity standard variation

# Create galaxy catalog and Cluster object
data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, "chang13", zsrc_min = cluster_z + 0.2, shapenoise=shapenoise, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
gc = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, data)

gc.compute_tangential_and_cross_components(geometry="flat")
radius = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)

# Create binning profile por binned methods
bin_edges = da.make_bins(0.7, 4, 50)
profile = gc.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
from clmm.support.sampler import samplers

# radius = convert_units(gc_noisy_z.galcat['theta'], 'radians', 'Mpc', redshift=gc_noisy_z.z, cosmo=cosmo)
logm_0 = random.uniform(13., 17., 1)[0]

logm_est = samplers['basinhopping'](shear_log_likelihood, logm_0, minimizer_kwargs={'args':(gc, radius, shapenoise, cosmo)})

print(logm_est)

In [ ]:
np.random.seed(0)

# Define cosmological parameters
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)
    
cluster_m     = 1.e15 # Cluster mass
cluster_z     = 0.4   # Cluster redshift
concentration = 4     # Concentrion parameter NFW profile
ngals         = 10000 # Number of galaxies
Delta         = 200   # Overdensity parameter definition NFW profile
cluster_ra    = 0.0   # Cluster right ascension
cluster_dec   = 0.0   # Cluster declination
shapenoise    = 1e-1 # True ellipticity standard variation

# Create galaxy catalog and Cluster object
data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, "chang13", zsrc_min = cluster_z + 0.2, shapenoise=shapenoise, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
gc = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, data)

gc.compute_tangential_and_cross_components(geometry="flat")
radius = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)

# Create binning profile por binned methods
bin_edges = da.make_bins(0.7, 4, 50)
profile = gc.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
from clmm.support.sampler import samplers

# radius = convert_units(gc_noisy_z.galcat['theta'], 'radians', 'Mpc', redshift=gc_noisy_z.z, cosmo=cosmo)
logm_0 = random.uniform(13., 17., 1)[0]

logm_est = samplers['basinhopping'](shear_log_likelihood, logm_0, minimizer_kwargs={'args':(gc, radius, shapenoise, cosmo)})

print(logm_est)

In [ ]:
np.random.seed(0)

# Define cosmological parameters
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)
    
cluster_m     = 1.e15 # Cluster mass
cluster_z     = 0.4   # Cluster redshift
concentration = 4     # Concentrion parameter NFW profile
ngals         = 10000 # Number of galaxies
Delta         = 200   # Overdensity parameter definition NFW profile
cluster_ra    = 0.0   # Cluster right ascension
cluster_dec   = 0.0   # Cluster declination
shapenoise    = 1 # True ellipticity standard variation

# Create galaxy catalog and Cluster object
data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, "chang13", zsrc_min = cluster_z + 0.2, shapenoise=shapenoise, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)
gc = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, data)

gc.compute_tangential_and_cross_components(geometry="flat")
radius = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)

# Create binning profile por binned methods
bin_edges = da.make_bins(0.7, 4, 50)
profile = gc.make_radial_profile("Mpc", bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True)

In [ ]:
from clmm.support.sampler import samplers

# radius = convert_units(gc_noisy_z.galcat['theta'], 'radians', 'Mpc', redshift=gc_noisy_z.z, cosmo=cosmo)
logm_0 = random.uniform(13., 17., 1)[0]

logm_est = samplers['basinhopping'](shear_log_likelihood, logm_0, minimizer_kwargs={'args':(gc, radius, shapenoise, cosmo)})

print(logm_est)